# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
# Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [7]:
# Your code here; use a subquery. No join is necessary 
cur.execute('''SELECT customerNumber, contactLastName, contactFirstName
               FROM customers
               WHERE customerNumber IN (SELECT customerNumber
                                        FROM orders
                                        WHERE orderDate = '2003-01-31')
                                        ;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [11]:
# Your code here
cur.execute('''SELECT productName, COUNT(orderNumber) AS numberOrdered, SUM(quantityOrdered) as totalUnitsSold
               FROM orderdetails
               JOIN products
               USING(productCode)
               GROUP BY productName
               ;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,productName,numberOrdered,totalUnitsSold
0,18th Century Vintage Horse Carriage,28,907
1,18th century schooner,27,1011
2,1900s Vintage Bi-Plane,28,940
3,1900s Vintage Tri-Plane,28,1009
4,1903 Ford Model A,27,883
...,...,...,...
104,The Mayflower,27,898
105,The Queen Mary,27,896
106,The Schooner Bluenose,27,934
107,The Titanic,27,952


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [16]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

cur.execute('''SELECT productName, COUNT(DISTINCT customerNumber) AS num_customer_purchases
               FROM products
               JOIN orderDetails
               USING(productCode)
               JOIN orders
               USING(orderNumber)
               GROUP BY productName
               ORDER BY num_customer_purchases DESC
               ;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,productName,num_customer_purchases
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [30]:
# Your code here
cur.execute("""SELECT DISTINCT employeeNumber, officeCode, o.city, firstName, lastName
               FROM employees e
               JOIN offices o
               USING(officeCode)
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders
               USING(customerNumber)
               JOIN orderDetails
               USING(orderNumber)
               WHERE productCode IN (SELECT productCode
                                            FROM products
                                            JOIN orderdetails
                                            USING(productCode)
                                            JOIN orders
                                            USING(orderNumber)
                                            GROUP BY productCode
                                            HAVING COUNT(DISTINCT customerNumber) < 20);
                                            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1501,7,London,Larry,Bott
2,1337,4,Paris,Loui,Bondur
3,1166,1,San Francisco,Leslie,Thompson
4,1286,3,NYC,Foon Yue,Tseng
5,1612,6,Sydney,Peter,Marsh
6,1611,6,Sydney,Andy,Fixter
7,1401,4,Paris,Pamela,Castillo
8,1621,5,Tokyo,Mami,Nishi
9,1323,3,NYC,George,Vanauf


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [31]:
# Your code here
cur.execute('''SELECT DISTINCT employeeNumber, firstName, lastName, COUNT(DISTINCT customerNumber) AS num_of_customers
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               GROUP BY employeeNumber
               HAVING AVG(creditLimit) > 15000
               ;''')
df=pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,num_of_customers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!